In [196]:
# DEPENDENCIES

import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# COIN API KEY (needs to remind private)

API_KEY = ""

with open("coinApiKey",'r') as f :
    API_KEY = f.read()


headers = { 'X-CoinAPI-Key': API_KEY}


# DESCRIPTION DATASET

def description(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values
    summary['PercMissing'] = df.isnull().sum().values / df.isnull().count().values
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.iloc[0].values
    summary['Second Value'] = df.iloc[1].values
    summary['Third Value'] = df.iloc[2].values
    return summary

# COLORS

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    ORANGE = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


In [197]:
# PARSING BINANCE DATA

# Export your file from binance order history, as they suck they export it as an old version of excel
# You have to convert it into a .csv file here for example https://convertio.co/fr/xlsx-csv/

# I always trade in USDT, and I once failed and transfer money to BUSD so I don't want these lignes

files = ["data.csv"]
sheets = ["sheet1"]
reference = "USDT"
nope = "BUSD"



df = pd.read_csv(files[0])

def clean(df):
    df = df.dropna()
    df = df[(df.status=="Filled") & (df.Pair.str.match(r'.*{}.*'.format(reference))==True)]
    df = df[(df.Pair.str.match(r'.*{}'.format(nope))==False)]
    #cleanDf = pd.to_timedate(df["Date(UTC)"])
    
    cleanDf =df[["Type","AvgTrading Price"]]
    
    cleanDf.Type = cleanDf.Type.apply(lambda x: 1 if x == "SELL" else 0)
    cleanDf["price"] = df["Order Price"].astype(float)
    cleanDf["amount"] = df["Order Amount"].astype(float)
    cleanDf["pair"] = df["Pair"].apply(lambda x : x.split(reference)[0])
    
    return cleanDf


def compute_change(group):
    return pd.DataFrame({
            'Total invested':(group[group.Type==0]["price"] * group[group.Type==0]["amount"]).sum(),
            'Total cashed': (group[group.Type==1]["price"] * group[group.Type==1]["amount"]).sum(),

            'Earning (dollar)': (group[group.Type==1]["price"] * group[group.Type==1]["amount"]).sum() -(group[group.Type==0]["price"] * group[group.Type==0]["amount"]).sum(),
            'Earning (crypto)':group[group.Type==0]["amount"].sum()-group[group.Type==1]["amount"].sum()
    }, index=[0])


interesting_df = clean(df).groupby("pair").apply(compute_change).reset_index()

/home/kkameleon/.local/lib/python3.9/site-packages/pandas/core/generic.py:5489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-197-dabbe839e4cb>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanDf["price"] = df["Order Price"].astype(float)
<ipython-input-197-dabbe839e4cb>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [198]:
# MARKET RATES

# DO NOT COMPUTE TOO MUCH TIME
#coinApi is free with 100 requests per day, not enough to trade hourly

rates =[]
for i in range(len(interesting_df.pair)):
    response = requests.get('https://rest.coinapi.io/v1/exchangerate/{}/USDT'.format(interesting_df.pair[i]), headers=headers)
    print(response.content)
    rates.append(response.json()['rate'])
interesting_df["Current rate"]=rates


b'{\n  "time": "2021-02-18T17:35:17.4650000Z",\n  "asset_id_base": "1INCH",\n  "asset_id_quote": "USDT",\n  "rate": 5.0287500\n}'
b'{\n  "time": "2021-02-18T17:35:17.6660000Z",\n  "asset_id_base": "ADA",\n  "asset_id_quote": "USDT",\n  "rate": 0.917849913095257940437875551\n}'
b'{\n  "time": "2021-02-18T17:35:17.7650000Z",\n  "asset_id_base": "BTC",\n  "asset_id_quote": "USDT",\n  "rate": 52057.694481302808914532933807\n}'
b'{\n  "time": "2021-02-18T17:35:14.4640000Z",\n  "asset_id_base": "BTT",\n  "asset_id_quote": "USDT",\n  "rate": 0.0012304\n}'
b'{\n  "time": "2021-02-18T17:35:17.4650000Z",\n  "asset_id_base": "DASH",\n  "asset_id_quote": "USDT",\n  "rate": 287.5700000\n}'
b'{\n  "time": "2021-02-18T17:35:17.8512137Z",\n  "asset_id_base": "DOT",\n  "asset_id_quote": "USDT",\n  "rate": 31.139501794376661182091914026\n}'
b'{\n  "time": "2021-02-18T17:35:18.1895526Z",\n  "asset_id_base": "ETH",\n  "asset_id_quote": "USDT",\n  "rate": 1918.8446375902586363220197571\n}'
b'{\n  "time": "

In [199]:
interesting_df["BONUS"]= interesting_df["Current rate"]*interesting_df["Earning (crypto)"]+interesting_df["Earning (dollar)"]
bonus = interesting_df["BONUS"].sum()
if bonus >=0 :
    print(bcolors.OKGREEN+"Yes, Your curent bonus is {} USDT".format(bonus)+bcolors.ENDC)
else : 
    print(bcolors.FAIL + "Aïe, Your curent bonus is {} USDT".format(bonus)+bcolors.ENDC)
interesting_df

Yes, Your curent bonus is 23.182780742965573 USDT


,pair,level_1,Total invested,Total cashed,Earning (dollar),Earning (crypto),Current rate,BONUS
0,1INCH,0,49.963500,0.000000,-49.963500,9.000000,5.028750,-4.704750
1,ADA,0,68.406784,0.000000,-68.406784,77.200000,0.917850,2.451229
2,BTC,0,49.969887,0.000000,-49.969887,0.001016,52057.694481,2.920730
3,BTT,0,10.999615,11.327877,0.328262,10.000000,0.001230,0.340566
4,DASH,0,59.998252,69.973960,9.975707,0.000250,287.570000,10.047600
5,DOT,0,49.962000,22.509730,-27.452270,1.000000,31.139502,3.687232
6,ETH,0,112.261883,113.819995,1.558112,0.000070,1918.844638,1.692431
7,FIL,0,11.008582,0.000000,-11.008582,0.260000,42.775200,0.112970
8,GRT,0,27.647900,0.000000,-27.647900,12.700000,2.306970,1.650619
9,IOTA,0,49.990013,0.000000,-49.990013,38.930000,1.456500,6.711532
